### Modelo OpenAI 

En este fichero aprendo a gestionar Langchain usando como modelo ChatGPT mediante su API. Trataré de conectar con Pinecone.

In [17]:
import os

import config

from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda

from langchain_openai import ChatOpenAI
from langchain_pinecone import PineconeVectorStore, PineconeEmbeddings
from pinecone import Pinecone

In [18]:
os.environ["OPENAI_API_KEY"] = config.OPENAI_API_KEY

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [19]:
response = model.invoke("Hola, ¿puedes confirmarme que estás funcionando?")
print(response.content)

¡Hola! Sí, estoy funcionando. ¿En qué puedo ayudarte hoy?


Ahora voy a usar el modelo tratando de recuperar los archivos almacenados en Pinecone

In [ ]:
# Modelo
llm = ChatOpenAI(model_name="gpt-4o-mini", api_key=config.OPENAI_API_KEY)

# Pinecone
pc = Pinecone(api_key=config.PINECONE_API_KEY)
index = pc.Index(config.INDEX_PINECONE)

embedding = PineconeEmbeddings(
    model=config.embedding_model,
    api_key=config.PINECONE_API_KEY
)

vector_store = PineconeVectorStore(embedding=embedding, index=index, text_key="body")

# Recuperar documentos de Pinecone para usarlos como contexto
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [ ]:
# Generar respuesta usando el modelo y los documentos recuperados

# Defino un prompt con el contexto, poniendo al bot en situación y haciendo la pregunta. Esto es una plantilla.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Responde solo con el contexto. Si falta info, dilo."),
    ("user", "Pregunta: {question}\n\nContexto:\n{context}")
])

# Formateo los documentos para el prompt. De momento no uso los metadatos de las noticias
def format_docs(docs):
    return "\n\n".join(f"• {d.page_content}" for d in docs)

# Construyo el pipeline para el RAG
rag = (
    RunnableParallel(
        context=retriever | RunnableLambda(format_docs),
        question=lambda x: x
    )
    | prompt
    | llm
)

# Hago la pregunta y obtengo la respuesta
query = "Putin conoce Roman?"

result = rag.invoke(query)
print(result)

content='Putin conocía a Roman Starovoit, ya que él era un exministro de Transporte que fue despedido por el presidente ruso antes de su muerte, la cual ha sido considerada sospechosa.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 3713, 'total_tokens': 3755, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C8XgS9TccBY1jQWFMwAlyoPx0H7uW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--79a372e4-8898-4a22-8f75-35a418f70219-0' usage_metadata={'input_tokens': 3713, 'output_tokens': 42, 'total_tokens': 3755, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


Cosas que quiero mejorar:
1. Tener conversación e historial de chat.
2. Gestionar los idiomas, hacer algo de prompt engineering.
3. Al recuperar documentos me gustaría poner en práctica añadir docs randoms si mejoran 35% la respuesta.
4. Aplicar cosillas de Advanced y Modular RAG.